In [1]:
from bs4 import BeautifulSoup
from multiprocessing import Process
from concurrent.futures import ThreadPoolExecutor
import requests
import threading
import pandas as pd
import time
import concurrent.futures
import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_rows', 5000)
pd.set_option('display.max_columns', 5000)
pd.set_option('display.width', -1)
import re

In [2]:
url = 'https://kontakt.az/'
content = requests.get(url)
soup = BeautifulSoup(content.text,'html.parser')

In [ ]:
st = time.time()
df = pd.DataFrame()
sub_category = []
category_links = []
paginated_links = []
products_links = []
df = pd.DataFrame()
prop = []
link = []
code = []
name = []
price = []
cash_price = []
category = []


def get_category_links(soup):
    for i in soup.find_all('a',class_='menuliparent_a'):
        if i.get('href') not in category_links:
                category_links.append(i.get('href'))
    return 


def pagination(category_links):
    for link in category_links:
        content_2 = requests.get(link)
        soup_2 = BeautifulSoup(content_2.text,'html.parser')
        try:
            b = soup_2.find('span',class_='pages').text.split(' / ')[1]
            a = soup_2.find('span',class_='pages').text.split(' / ')[0]
        except:
            b = 1
            a = 1
        for i in range(int(a),int(b)+1):
            paginated_links.append(link + 'page/' + str(i) + '/')
    return 


def get_prod_links(paginated_links):
    for urls in paginated_links:
        content_3 = requests.get(urls)
        soup_3 = BeautifulSoup(content_3.text,'html.parser')
        a = soup_3.find_all('a',draggable='false')
        for url in a:
            products_links.append(url.get('href'))
    return


def get_prod_details(products_links):
    for i in products_links:
        e = requests.get(i)
        s = BeautifulSoup(e.text,'html.parser')
        link.append(i)
        code.append(s.find('div',class_='product-status').find('span').text)
        name.append(s.find('h1',class_='title').text)
        price.append(s.find('span',class_='nprice').text)
        category.append(s.find('div',class_='col-12').find_all('a')[1].text)
        sub_category.append(s.find('div',class_='col-12').find_all('a')[2].text)
        try:
            pr = re.findall('\d+',s.find('div',class_='discounted-price').text)
            cash_price.append(float(pr[0])+float(pr[1])/100)
        except:
            pr = None
            cash_price.append(pr)
        pc = s.find('div',class_='product-status').find('span').text
        c = s.find_all('ul',class_='features-review-tabs submenuItems_params')[0].text
        v = s.find_all('ul',class_='features-review-tabs submenuItems_params')[1].text
        c = c.replace('\n\n','').replace('\n ','').replace('\n',',').split(',')
        v = v.replace('\n\n','').replace('\n ','').replace('\n',',').split(',')
        sd = dict(zip(c,v))
        prop.append(sd)
    return


def details_convert_to_DataFrame(category, sub_category, code, link, name, price, cash_price, prop):
    df['Category'] = category
    df['Sub_Category'] = sub_category
    df['Code'] = code
    df['Link'] = link
    df['Name'] = name
    df['Price'] = price
    df['Cash _Price'] = cash_price
    df['Properties'] = prop
    return df


def main():
    get_category_links(soup)
    category_links.remove('https://kontakt.az/telefonlar/mobil-telefonlar/xiaomi-mobil-telefonlar/')
    category_links.remove('https://kontakt.az/telefonlar/mobil-telefonlar/apple-mobil-telefonlar/')
    category_links.remove('https://kontakt.az/telefonlar/mobil-telefonlar/samsung-mobil-telefonlar/')
    pagination(category_links)
    get_prod_links(paginated_links)
    get_prod_details(products_links)
    details_convert_to_DataFrame(category,sub_category,code,link,name,price,cash_price,prop)
    return

    
if __name__ == '__main__':
    main()

    
et = time.time()
result = et - st
df.to_excel('KontaktHome.xlsx')
print(f'Code Run Time: {result}')

# Below codes may not work properly

In [25]:
df = pd.DataFrame()
sub_category = []
category_links = []
paginated_links = []
products_links = []
df = pd.DataFrame()
prop = []
link = []
code = []
name = []
price = []
cash_price = []
category = []


def get_category_links(soup):
    for i in soup.find_all('a',class_='menuliparent_a'):
        if i.get('href') not in category_links:
                category_links.append(i.get('href'))
    return category_links


def pagination(category_links):
    for link in category_links:
        content_2 = requests.get(link)
        soup_2 = BeautifulSoup(content_2.text,'html.parser')
        try:
            b = soup_2.find('span',class_='pages').text.split(' / ')[1]
            a = soup_2.find('span',class_='pages').text.split(' / ')[0]
        except:
            b = 1
            a = 1
        for i in range(int(a),int(b)+1):
            paginated_links.append(link + 'page/' + str(i) + '/')
    return paginated_links


def get_prod_links(paginated_links):
    for urls in paginated_links[1:9]:
        content_3 = requests.get(urls)
        soup_3 = BeautifulSoup(content_3.text,'html.parser')
        a = soup_3.find_all('a',draggable='false')
        for url in a:
            products_links.append(url.get('href'))
    return products_links


def get_prod_details(products_links):
    for i in products_links[1:10]:
        e = requests.get(i)
        s = BeautifulSoup(e.text,'html.parser')
        link.append(i)
        code.append(s.find('div',class_='product-status').find('span').text)
        name.append(s.find('h1',class_='title').text)
        price.append(s.find('span',class_='nprice').text)
        category.append(s.find('div',class_='col-12').find_all('a')[1].text)
        sub_category.append(s.find('div',class_='col-12').find_all('a')[2].text)
        try:
            pr = re.findall('\d+',s.find('div',class_='discounted-price').text)
            cash_price.append(float(pr[0])+float(pr[1])/100)
        except:
            pr = None
            cash_price.append(pr)
        pc = s.find('div',class_='product-status').find('span').text
        c = s.find_all('ul',class_='features-review-tabs submenuItems_params')[0].text
        v = s.find_all('ul',class_='features-review-tabs submenuItems_params')[1].text
        c = c.replace('\n\n','').replace('\n ','').replace('\n',',').split(',')
        v = v.replace('\n\n','').replace('\n ','').replace('\n',',').split(',')
        sd = dict(zip(c,v))
        prop.append(sd)
    return prop


def details_convert_to_DataFrame(category, sub_category, code, link, name, price, cash_price, prop):
    df['Category'] = category
    df['Sub_Category'] = sub_category
    df['Code'] = code
    df['Link'] = link
    df['Name'] = name
    df['Price'] = price
    df['Cash _Price'] = cash_price
    df['Properties'] = prop
    return df


def main():
    t1 = Process(target=get_category_links,args=(soup,))
#     category_links.remove('https://kontakt.az/telefonlar/mobil-telefonlar/xiaomi-mobil-telefonlar/')
#     category_links.remove('https://kontakt.az/telefonlar/mobil-telefonlar/apple-mobil-telefonlar/')
#     category_links.remove('https://kontakt.az/telefonlar/mobil-telefonlar/samsung-mobil-telefonlar/')
    t2 = Process(target=pagination,args=(category_links,))
    t3 = Process(target=get_prod_links,args=(paginated_links,))
    t4 = Process(target=get_prod_details,args=(products_links,))
    t5 = Process(target=details_convert_to_DataFrame,args=(category,sub_category,code,link,name,price,cash_price,prop))
    t1.start()
    t1.join()
    t2.start()
    t2.join()
    t3.start()
    t3.join()
    t4.start()
    t4.join()
    return category_links, paginated_links, products_links, prop, df

    
if __name__ == '__main__':
    st = time.time()
    main()
    et = time.time()
    result = et - st
    print(f'Code Run Time: {result}')

RecursionError: maximum recursion depth exceeded

In [3]:
df = pd.DataFrame()
sub_category = []
category_links = []
paginated_links = []
products_links = []
df = pd.DataFrame()
prop = []
link = []
code = []
name = []
price = []
cash_price = []
category = []


def get_category_links(soup):
    for i in soup.find_all('a',class_='menuliparent_a'):
        if i.get('href') not in category_links:
                category_links.append(i.get('href'))
    return


def pagination(category_links):
    for link in category_links:
        content_2 = requests.get(link)
        soup_2 = BeautifulSoup(content_2.text,'html.parser')
        try:
            b = soup_2.find('span',class_='pages').text.split(' / ')[1]
            a = soup_2.find('span',class_='pages').text.split(' / ')[0]
        except:
            b = 1
            a = 1
        for i in range(int(a),int(b)+1):
            paginated_links.append(link + 'page/' + str(i) + '/')
    return 


def get_prod_links(paginated_links):
    for urls in paginated_links[1:9]:
        content_3 = requests.get(urls)
        soup_3 = BeautifulSoup(content_3.text,'html.parser')
        a = soup_3.find_all('a',draggable='false')
        for url in a:
            products_links.append(url.get('href'))
    return 


def get_prod_details(products_links):
    for i in products_links[1:10]:
        e = requests.get(i)
        s = BeautifulSoup(e.text,'html.parser')
        link.append(i)
        code.append(s.find('div',class_='product-status').find('span').text)
        name.append(s.find('h1',class_='title').text)
        price.append(s.find('span',class_='nprice').text)
        category.append(s.find('div',class_='col-12').find_all('a')[1].text)
        sub_category.append(s.find('div',class_='col-12').find_all('a')[2].text)
        try:
            pr = re.findall('\d+',s.find('div',class_='discounted-price').text)
            cash_price.append(float(pr[0])+float(pr[1])/100)
        except:
            pr = None
            cash_price.append(pr)
        pc = s.find('div',class_='product-status').find('span').text
        c = s.find_all('ul',class_='features-review-tabs submenuItems_params')[0].text
        v = s.find_all('ul',class_='features-review-tabs submenuItems_params')[1].text
        c = c.replace('\n\n','').replace('\n ','').replace('\n',',').split(',')
        v = v.replace('\n\n','').replace('\n ','').replace('\n',',').split(',')
        sd = dict(zip(c,v))
        prop.append(sd)
    return 


def details_convert_to_DataFrame(category, sub_category, code, link, name, price, cash_price, prop):
    df['Category'] = category
    df['Sub_Category'] = sub_category
    df['Code'] = code
    df['Link'] = link
    df['Name'] = name
    df['Price'] = price
    df['Cash _Price'] = cash_price
    df['Properties'] = prop
    return df


def main():
    with ThreadPoolExecutor(max_workers=8) as executor:
        executor.map(get_category_links, soup)
        executor.map(pagination, category_links)
#         executor.map(get_prod_links, paginated_links)
#         executor.map(get_prod_details, products_links)
#         executor.map(details_convert_to_DataFrame,category,sub_category,code,link,name,price,cash_price,prop)
    return

    
if __name__ == '__main__':
    st = time.time()
    main()
    et = time.time()
    result = et - st
    print(f'Code Run Time: {result}')

Code Run Time: 0.012087345123291016


In [4]:
print(len(category_links))
print(len(paginated_links))
print(len(products_links))
print(len(prop))
print(len(df))

109
0
0
0
0


In [5]:
df = pd.DataFrame()
prop = []
link = []
code = []
name = []
price = []
cash_price = []
category = []
sub_category = []
for i in products_links[1:30]:
    e = requests.get(i)
    s = BeautifulSoup(e.text,'html.parser')
    link.append(i)
    code.append(s.find('div',class_='product-status').find('span').text)
    name.append(s.find('h1',class_='title').text)
    price.append(s.find('span',class_='nprice').text)
    category.append(s.find('div',class_='col-12').find_all('a')[1].text)
    sub_category.append(s.find('div',class_='col-12').find_all('a')[2].text)
    try:
        pr = re.findall('\d+',s.find('div',class_='discounted-price').text)
        cash_price.append(float(pr[0])+float(pr[1])/100)
    except:
        pr = None
        cash_price.append(pr)
    pc = s.find('div',class_='product-status').find('span').text
    c = s.find_all('ul',class_='features-review-tabs submenuItems_params')[0].text
    v = s.find_all('ul',class_='features-review-tabs submenuItems_params')[1].text
    c = c.replace('\n\n','').replace('\n ','').replace('\n',',').split(',')
    v = v.replace('\n\n','').replace('\n ','').replace('\n',',').split(',')
    sd = dict(zip(c,v))
    prop.append(sd)
df = pd.DataFrame(category,columns=['Category'])
df['Sub_Category'] = sub_category
df['Code'] = code
df['Link'] = link
df['Name'] = name
df['Price'] = price
df['Discounted_Price'] = cash_price
df['Properties'] = prop